In [ ]:
import pickle
import pandas as pd
import os
import peppi_py
import tree
import json
import io
import math
import tqdm
import functools

from slippi_db import utils, preprocessing, parse_peppi

In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

In [ ]:
root = '/Users/vlad/SSBM/Replays/'

In [ ]:
parsed_path = os.path.join(root, 'parsed.pkl')

with open(parsed_path, 'rb') as f:
    data_bytes = f.read()
len(data_bytes)

In [ ]:
%%time
data = pickle.loads(data_bytes)
del data_bytes
len(data)

In [ ]:
# %%time
# data_json = json.dumps(data)
# len(data_json)

In [ ]:
# %%time
# json.loads(data_json);

In [ ]:
df = pd.DataFrame(data)

In [ ]:
@functools.cache
def compact_raw(raw: str) -> str:
    if 'Mango' in raw:
        return 'Mango'
    if raw.startswith('Players/'):
        return raw.split('/')[1]
    return raw

df['compact_raw'] = df['raw'].map(compact_raw)

In [ ]:
df.groupby('compact_raw')['reason'].value_counts()

In [ ]:
df.groupby('compact_raw')['is_training'].mean()

In [ ]:
not_training_reasons = df[df['not_training_reason'] != '']
not_training_reasons.groupby('compact_raw')['not_training_reason'].value_counts()

In [ ]:
df['has_winner'] = ~df['winner'].isnull()
valid = df[df['valid']]
training = df[df['is_training'] == True]
no_winner = training[~training['has_winner']]

In [ ]:
training.groupby('compact_raw')['has_winner'].mean()

In [ ]:
import typing as tp
import itertools
import melee
from slippi_ai import nametags

In [ ]:
%%time

def split_row(row: dict) -> tp.Iterator[dict]:
    for player in row['players']:
        new_row = row.copy()
        del new_row['players']
        new_row.update(player)
        name = nametags.name_from_metadata(player)
        new_row['name'] = nametags.normalize_name(name)
        yield new_row

per_player_data = []
for row in tqdm.tqdm_notebook(data):
    if not row['valid'] or not row['is_training']:
        continue
    per_player_data.extend(list(split_row(row)))

per_player_df = pd.DataFrame(per_player_data)

In [ ]:
def char_to_str(char: int) -> str:
    return melee.Character(char).name

per_player_df['char'] = per_player_df['character'].map(char_to_str)

In [ ]:
per_player_df['compact_raw'] = per_player_df['raw'].map(compact_raw)

In [ ]:
# per_player_df.groupby('compact_raw')['name'].value_counts()

In [ ]:
zain_df = per_player_df[per_player_df['name'] == 'Zain']
zain_df['compact_raw'].value_counts()

In [ ]:
per_player_df['netplay.name'].value_counts()

In [ ]:
legal_df = per_player_df[per_player_df['compact_raw'] != 'Mango']

In [ ]:
x = legal_df.groupby(['char', 'name']).size().reset_index(name='count')
x = x.sort_values(by='count', ascending=False).groupby('char').head(10)

In [ ]:
x[x['char'] == 'YOSHI']

In [ ]:
per_player_df['char'].value_counts()